In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
tf.__version__

'2.7.0'

***Part1 - Data Preprocessing***

In [4]:
dataset=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Churn_Modelling.csv')
X=dataset.iloc[:,3:-1].values
y=dataset.iloc[:,-1].values
print("x", X)
print("y",y)

x [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y [1 0 1 ... 1 1 0]


*Encoding Categorical Data*

--- Label encoding Gender column

In [5]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2]) 
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


-- One Hot Encoding Geography column

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct= ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X=np.array(ct.fit_transform(X))

In [20]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


*Splitting the dataset into Training and Test set*

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

*Feature Scaling*

In [8]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

***Part 2 - Building ANN***

*Initializing an ANN*

In [9]:
ann = tf.keras.models.Sequential()

*Add Input layer and first Hiddem layer in the network*

In [11]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Add second hidden layer in the network

In [12]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding the output

In [13]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

***Part3 - Training the ANN***

*Compiling the ANN*

In [15]:
ann.compile(optimizer= 'adam' ,loss= 'binary_crossentropy' ,metrics= ['accuracy'])

*Training ANN on training dataset*

In [16]:
ann.fit(X_train,y_train,batch_size= 32, epochs= 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5067 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4598 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4423 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4345 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4301 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4268 - accuracy: 0.7966
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4245 - accuracy: 0.8096
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4217 - accuracy: 0.8144
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4197 - accuracy: 0.8188
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4176 - accura

***Part 4 - Making predictions and evaluating the Model***

*Predict the result of single observation*


Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

# So, should we say goodbye to that customer?






In [19]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

[[False]]


*Predicting Test Set results*

In [20]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


*Making the Confusion Matrix*

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1543   52]
 [ 266  139]]


0.841

**We got 84% accurracy (1543 is the number customer leaves the bank and 139 customer stays in the bank)**